In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import scipy as sp
from tqdm import tqdm

# I) Investigating Diabetes Trends in the Synthea Data

In this module, we will:
1. Load and explore synthetic EHR data generated using Synthea.
2. Identify patients diagnosed with Type 2 Diabetes using SNOMED CT codes.
3. Analyze Hemoglobin A1C (HbA1c) levels in diabetic and non-diabetic patients.
4. Visualize data distributions and perform statistical testing.
5. Discuss findings and explore possible extensions.

## I.a) Loading of Relevant Data
Data for 1,000 synthetic patients, divided into four states, was generated using the [Synthea tool](https://synthea.mitre.org/)
This data is split into four separate directories
```
output_hi/ # patients in Hawaii
output_ma/ # patients in Massacusetts
output_tx/ # patients in Texas
output_wa/ # patients in Washington
```

In part 1 of this two workshop series, we worked exclusively with the patient demographics data stored in patients.parquet
For the subsequent investigation, we will be relying on data stored across three new files:
- `conditions.parquet`: Stores SNOMED CT coded patient conditions and diagnoses
- `observations.parquet`: Includes vital signs and LOINC coded lab results for all patients
- `medications.parquet`: Tracks RxNorm coded medication prescribing and dispensing/fill data for all patients \
Below is a convenience function that can be used to load the concatenated conditions, observations, and medications for all states
into a single data frame

In [ ]:
def load_data_for_file(filename):
    print(f"Loading data for {filename}")
    url_prefix = "https://github.com/expmed/arch_workshop_data_wrangling2_ws10/raw/refs/heads/main/data/"
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        pd.read_parquet(f"{url_prefix}{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi_small', 'output_ma_small', 'output_tx_small', 'output_wa_small']) # loop over each output directory
    ])
    return df

In [ ]:
# load in the patients, conditions, observations, and medications data
patients = load_data_for_file('patients.parquet')
conditions = load_data_for_file('conditions.parquet')
observations = load_data_for_file('observations.parquet')
medications = load_data_for_file('medications.parquet')

In [ ]:
patients

In [ ]:
conditions

In [ ]:
observations

In [ ]:
medications

## I.b) Selecting Patients with Type 2 Diabetes Diagnoses
For this investigation, we are interested in patients who have been diagnosed with Type 2 Diabetes \
In order to do this, we filter the conditions table to get all diagnoses of type 2 diabetes \
We can use the SNOMED CT code for type 2 diabetes: `44054006`

In [ ]:
diabetes = conditions.query('CODE == 44054006') # we could also use a substring search if we didn't know the code

In [ ]:
diabetes

In [ ]:
# count the unique patients with Type 2 diabetes 
# (NOTE: Not always the same as the number shown above for rows, as patients will often have repeat diagnoses depending on the condition/disorder)
diabetes['PATIENT'].nunique()

## I.c) Analysis of Hemoglobin A1c (HbA1c) Labs in Type 2 and Non-Type 2 patients
Now we will analyze Hemoglobin A1C levels for patients with and without a diabetes diagnosis \
To do this, we first filter the observations table for all hemoglobin A1C lab results \
We can use LOINC code: `4548-4`

In [ ]:
hemoglobin_a1c = observations[observations['CODE'] == '4548-4']

In [ ]:
hemoglobin_a1c

Next we will add another variable/column to the hemoglobin_a1c data to label patients with/without a Type 2 diabetes diagnosis
Here we are performing multiple operations in a single line:
1. The outermost operation is a call to `hemoglobin_a1c.assign()` to assign a new variable/column to the dataframe of HbA1c labs
2. To the new variable `HASDIABETES`, we are assigning a series
3. The series we are assigning is a boolean series created by calling the `.isin()` method, effectively asking if the `PATIENT` ID of the record is in the set of IDs in the type 2 diabetes diagnoses
4. On this boolean series (which by default has values `True` and `False`) we are transforming the values into integers (1 for True, 0 for False) using `.astype('int')` to convert to integers

*NOTE:* We are cheating/simplifying somewhat. Really what we probably should do is label the HbA1c labs based on whether they occured before/after the diagnosis of Type 2 diabetes (within a certain time delta) to perhaps get a more accurate labeling of the labs. However, this would require creating an auxiliary data structure and lead to more complicated code, so we use this simpler approach for sake of brevity.

In [ ]:
hemoglobin_a1c_labeled = hemoglobin_a1c.assign(
    HASDIABETES=hemoglobin_a1c['PATIENT'].isin(diabetes['PATIENT']).astype('int')
)

In [ ]:
hemoglobin_a1c_labeled

Now we will plot the distributions of A1C in the two groups using the Seaborn package \
and display summary statistics for both

In [ ]:
# plot a Kernel Density Estimate of the distrubtion (probability density function) of Hemoglobin A1C lab measurements for Type 2 and Non Type 2 Patients
sns.kdeplot(hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'), label="With Diabetes", color="red", alpha=0.4, common_norm=False)
sns.kdeplot(hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'), label="Without Diabetes", color="blue", alpha=0.4, common_norm=False)
# add a legend
plt.legend()
# change label of the xaxis
plt.xlabel('HbA1C (%)')
plt.show()

In [ ]:
# use seaborn to create a violin plot of the two distributions
sns.violinplot(
    hemoglobin_a1c_labeled.astype({'VALUE': float}),
    x='HASDIABETES',
    y='VALUE'
)
# change the label of the y-axis
plt.ylabel('HbA1C (%)')
plt.show()

The U.S. CDC has the following reference values published for Hemoglobin A1C

Normal: below 5.7% \
Prediabetes: 5.7% to 6.4% \
Diabetes: 6.5% or above 

In looking at the distribution plots above, we see that patients without a Type 2 diabetes diagnosis have an A1c distribution that is bi-modal, with one mode below 5.7% and another within the Pre-diabetes range of 5.7% to 6.4%

We also see that the patients with Type 2 diabetes diagnosis have a multi-modal HbA1c distribution, and a long tail that extends as high as to roughly 12%

In [ ]:
hemoglobin_a1c_labeled.astype({'VALUE': 'float'}).groupby('HASDIABETES')['VALUE'].describe()

If we want to quantify whether there is a significant difference in the two distributions \
one option is the use the Mann-Whitney U (Wilcoxon rank-sum) test \
provided by scipy's `mannwhitneyu()` function.

Another test we can perform is the Kolmogorov Smirnov test to test for subtle differences in shape of the two distributions.
We can perform a two-sample KS test using Scipy's `ks_2samp()` function.

The Mann-Whitney U test is mostly sensitive to changes in the median.
The KS test is sensitive to any differences in the two distributions (shape, spread, or median)

In [ ]:
from scipy.stats import mannwhitneyu
from scipy.stats import ks_2samp

In [ ]:
U1, p = mannwhitneyu(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'),
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'), 
    method="auto"
)

In [ ]:
print(f"U1: {U1}, p-value: {p}")

In [ ]:
statistic, p_value = ks_2samp(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'),
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float')
)

In [ ]:
print(f"KS Statistic: {statistic}, p-value: {p_value}")

# Discussion Points
1. Why is it that we observe a bi-modal distribution in A1C values among patients without a Type 2 Diabetes diagnosis?
2. Why do we observe right skewness and high A1C outliers in diabetic patients, while the medians of the two distributions are statistically close? What biological or clinical factors might explain this?
3. Why do we observe modes of the HbA1c distribution within the normal range for Type 2 diabetes patients?
4. How can we extend this analysis to add additional nuance (additional lab tests, adding data from other tables, partitioning patients further based on \
   other diagnoses, medications, demographics etc)?

# II. Open Ended Exploration & Extensions to Initial Analysis
Try to see if you can extend the initial analysis that we did on Hemoglobin A1C and diabetes

1. Are there any other labs that you could pull data for and compare distributions for between Type-2 and non-Type2 patients
2. Could you further subdivide the Type 2 and non-Type 2 patients into additional subsets based on other diagnoses/demographics/medication status?


**Click Below for Hints**

<details>
    <summary>Hint 1</summary>
    <p>The medications table has both a <code>'REASONCODE'</code> abd <code>'REASONDESCRIPTION'</code> column which describe why the medication was prescribed</p>
</details>
<details>
    <summary>Hint 2</summary>
    <p>If you aren't sure which LOINC to query for a particular lab for, you can always use the <code>Series.str.contains('_PATTERN_', case=False)</code> technique to perform a string search</p>
</details>
<details>
    <summary>Hint 3</summary>
    <p>If you want to search for the SNOMED CT code for other diagnoses, such as Pre-diabetes, you can do that via <a href="https://browser.ihtsdotools.org/?perspective=full&conceptId1=9414007&edition=MAIN/2025-06-01&release=&languages=en">SNOMED Browser</a></p>
    <p> You can also take advantage of string/keyword search, again via <code>Series.str.contains('_PATTERN_', case=False)</code></p>
</details>